# Réservations d'hôtels

Ce dataset fournit des données sur des réservations d'hôtels, telles que les dates d'arrivée et de départ, le nombre de
nuits passées, le nombre de clients, les annulations etc...
L'objectif ici est de construire un modèle d'apprentissage machine capable de prédire les probabilités d'une annulation
pour une commande donnée.

### Importation des bibliothèques

In [1]:
import pandas as pd
import seaborn as sns; sns.set()
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder as LE
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.svm import LinearSVC as LSVC
from sklearn.svm import SVC

from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc, plot_roc_curve

## 1 - Exploration des données

### Importation du dataset

On affecte le dataset à une variable de type DataFrame. </br>
On peut voir que le DataFrame contient 119390 lignes pour 32 colonnes.

In [2]:
df = pd.read_csv("./hotel_bookings.csv")
df.shape

FileNotFoundError: [Errno 2] No such file or directory: './hotel_bookings.csv'

### Affichage des informations du DataFrame

On observe ici une description des informations du DataFrame. </br>
Le type de chaque colonne est affiché ainsi que le nombre d'occurences de valeurs non nulle.
Le total de chaque type présent est aussi décrit.

In [ ]:
df.info()

### Statistiques descriptives

Ce tableau affiche pour chaque colonne le total d'occurences, la moyenne, la médiane, les valeurs minimum et maximum
ainsi que les quartiles.

In [ ]:
df.describe()

## 2 - Dataviz

In [ ]:
sns.set(style = "darkgrid")
plt.title("Canceled or not", fontdict = {'fontsize': 20})
ax = sns.countplot(x = "is_canceled", hue = 'hotel', data = df)

In [ ]:
plt.figure(figsize=(15,10))
sns.boxplot(y="arrival_date_week_number", x="hotel", data=df, hue="is_canceled", palette='Set1')

### Heatmap de corrélation

La heatmap de corrélation permet de représenter la corrélation pour chaque paire de colonnes à l'aide d'un code couleur.
On observe ici aucune corrélation particulière entre les données.

In [ ]:
corr = df.corr()
sns.heatmap(corr)

In [ ]:
# Un échantillon des données présentes dans le DataFrame
df.sample(10)

### Gestion des NA
Affichage de la somme des valeurs nulles pour chaque colonnes

In [ ]:
df.isna().sum()

Calcul du ratio de valeurs nulles pour chaque colonnes

In [ ]:
ratio_company = (df['company'].isna().sum() / len(df)) * 100
ratio_agent = (df['agent'].isna().sum() / len(df)) * 100
ratio_country = (df['country'].isna().sum() / len(df)) * 100

print("Company null ratio : " + str(ratio_company) + "\n" +
      "Agent null ratio : " + str(ratio_agent) + "\n" +
      "Country null ratio : " + str(ratio_country))

On observe que les colonnes company et agent ont un pourcentage important de valeurs nulles, c'est pourquoi
nous allons les retirer du dataset

In [ ]:
df = df.drop(columns=['company', 'agent'])

On élimine ensuite les lignes contenant des valeurs nulles

In [ ]:
df = df.dropna()
df.shape

On revérifie la présence de valeurs nulles dans le DataFrame. On voit qu'il n'en reste aucune

In [ ]:
df.isna().sum()

In [ ]:
df.info()

### Transformation des variables catégoriques en variables quantitatives

In [ ]:
# Conversion grâce à la méthode astype()
df['arrival_date_month'] = df['arrival_date_month'].astype('category').cat.codes

In [ ]:
# Utilisation d'un LabelEncoder

le = LE()
df['hotel'] = le.fit_transform(df['hotel'])
df['country'] = le.fit_transform(df['country'])
df['market_segment'] = le.fit_transform(df['market_segment'])
df['distribution_channel'] = le.fit_transform(df['distribution_channel'])
df['reserved_room_type'] = le.fit_transform(df['reserved_room_type'])
df['assigned_room_type'] = le.fit_transform(df['assigned_room_type'])
df['deposit_type'] = le.fit_transform(df['deposit_type'])
df['customer_type'] = le.fit_transform(df['customer_type'])
df['reservation_status'] = le.fit_transform(df['reservation_status'])
df['reservation_status_date'] = le.fit_transform(df['reservation_status_date'])

In [ ]:
# Suppression de la colonne meal car non-pertinente pour l'objectif voulu
df = df.drop(columns=['meal'])
df.info()

Observation de la corrélation des colonnes par rapport à is_canceled

In [ ]:
cancel_corr = df.corr()["is_canceled"]
cancel_corr.abs().sort_values(ascending=False)[1:]

### Séparation features/target

In [ ]:
y = df['is_canceled']
x = df.drop(columns=['is_canceled'])

On sépare nos données d'entrainement et de test avec la fonction train_test_split()

In [ ]:
splits = train_test_split(x, y, random_state=3, stratify=y)
x_tr, x_te, y_tr, y_te = splits

In [ ]:
# Vérification de la correspondance des dimension
for each in [x_tr, x_te, y_tr, y_te]:
    print(each.shape)

### Pipeline
Création d'une pipeline et d'une GridSearchCV afin de trouver la meilleures accuracy parmis plusieurs algorithme et
plusieurs jeux d'hyperparamètres

In [ ]:
pipeline_details = [('pca', PCA()),
                    ('knn', KNN())]

pipeline_svc = [('svc', SVC())]

pipeline = Pipeline(pipeline_details)

In [ ]:
hp_parameters = {'knn__n_neighbors': [1, 3, 5, 9, 21]}
hp_parameters_svc = {'svc__kernel': ['linear', 'poly', 'rbf', 'sigmoid']}

In [ ]:
hp_search = GridSearchCV(pipeline,
                         hp_parameters,
                         scoring='accuracy',
                         cv=5)
hp_search.fit(x_tr, y_tr)

In [ ]:
# Not working
# hp_search_svc = GridSearchCV(SVC(), hp_parameters_svc, scoring='accuracy', cv=5)
# hp_search_svc.fit(x_tr, y_tr)

In [ ]:
# Affichage du meilleur score trouvé
hp_search.best_score_


In [ ]:
# Affichage de meilleurs paramètres et algo
hp_search.best_params_


In [ ]:
# hp_search_svc.best_score_

Le résultat de la GridSearchCV nous indique que l'algorithme KNN avec l'hyperparamètre n_neighbors à 1

### Réajustement des features
Un peu de feature engineering pour essayer d'améliorer le score d'accuracy

In [ ]:
# Regroupemement des features stays_in_weekend_nights et feature engineering
total_nights = df["stays_in_weekend_nights"]+ df["stays_in_week_nights"]

df['total_nights'] = total_nights
# Regroupement des features adults, children et babies
total_people = df['adults'] + df['children'] + df['babies']
df['total_people'] = total_people

In [ ]:
# On supprime les colonnes qui ont servi aux regroupements
df = df.drop(columns=['adults', 'babies', 'children', 'stays_in_week_nights', 'stays_in_weekend_nights', 'deposit_type'])

In [ ]:
df.info()

In [ ]:
# Revérification de la heatmap de corrélation, on n'observe toujours pas de corrélation particulière
new_corr = df.corr()
sns.heatmap(new_corr)

In [ ]:
new_cancel_corr = df.corr()["is_canceled"]
new_cancel_corr.abs().sort_values(ascending=False)[1:]

In [ ]:
df = df.drop(columns=['lead_time', 'country'])

In [ ]:
df.info()

### Tests des modèles avec les nouvelles features

In [ ]:
y = df['is_canceled']
x = df.drop(columns=['is_canceled'])

In [ ]:
x_tr, x_te, y_tr, y_te = train_test_split(x, y, random_state=3, stratify=y, train_size=0.8)

In [ ]:
pipeline_details = [('pca', PCA()),
                    ('knn', KNN())]

In [ ]:
pipeline = Pipeline(pipeline_details)

In [ ]:
hp_parameters = {'knn__n_neighbors': [1, 3, 5, 9, 21]}

In [ ]:
hp_search = GridSearchCV(pipeline,
                         hp_parameters,
                         scoring='accuracy',
                         cv=5)
hp_search.fit(x_tr, y_tr)

In [ ]:
hp_search.best_score_


In [ ]:
hp_search.best_params_

On peut voir qu'avec les nouvelles features le score d'accuracy est passé de 90% à 96%

### Construction du modèle

In [ ]:
def get_accuracy(preds, target):
    m = target.shape[0] # nombre d'exemples
    total_correct = (preds == target).sum()
    accuracy = total_correct / m
    return str(round(accuracy * 100, 2)) + "%"

In [ ]:
knn = KNN(n_neighbors=1)

In [ ]:
knn.fit(x_tr, y_tr)

In [ ]:
preds = knn.predict(x_te)

In [ ]:
get_accuracy(preds, y_te)

### Création d'un PCA

In [ ]:
pca = PCA()
pca.fit(x_tr)

In [ ]:
x_tr_pca = pca.transform(x_tr)
x_te_pca = pca.transform(x_te)

In [ ]:
cumsum = np.cumsum(pca.explained_variance_ratio_)
sns.lineplot([i for i in range(1, len(cumsum) + 1)], cumsum)
plt.xlabel('Nombre de composante principale gardée')
plt.ylabel('Cumulative Explained Variance Ratio') # Ratio d'Informations (==variance) Conservees
plt.title('Graphe pour choisir le nombre de PC')

In [ ]:
pca = PCA(n_components=5)
pca.fit(x_tr)
x_tr_PCA = pca.transform(x_tr)
x_te_PCA = pca.transform(x_te)

In [ ]:
knn = KNN(n_neighbors=1)
knn.fit(x_tr_PCA, y_tr)

In [ ]:
preds = knn.predict(x_te_PCA)
get_accuracy(preds, y_te)

### Confusion matrix

In [ ]:
knn.fit(x_tr, y_tr)

In [ ]:
def show_cm(cm, labels):
    df_cm = pd.DataFrame(cm, labels, labels)
    sns.heatmap(df_cm, annot=True)
    plt.show()

In [ ]:
y_preds = knn.predict(x_te)
y_preds.shape

In [ ]:
cm_train = confusion_matrix(y_preds, y_te, labels=[0,1])
show_cm(cm_train, [0,1])
#Accuracy du training
accuracy = lambda p, y : (p==y).sum()/len(y)
print('Accuracy : {}'.format(accuracy(y_preds, y_te)))

### ROC / AUC

In [ ]:
proba = knn.predict_proba(x_tr)
proba1 = knn.predict_proba(x_te)
n_classes = 2
fpr = dict()
tpr = dict()
roc_auc = dict()

In [ ]:
try:
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_te, proba1[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
except ValueError as err:
    print(err)

In [ ]:
fpr["micro"], tpr["micro"], _ = roc_curve(y_te.ravel(), proba1[:, i].ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

In [ ]:
plt.figure()
lw = 2

In [ ]:
plt.plot(fpr[1], tpr[1], color='darkorange', lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[1])
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

In [ ]:
try:
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_tr, proba[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
except ValueError as err:
    print(err)

In [ ]:
fpr["micro"], tpr["micro"], _ = roc_curve(y_tr.ravel(), proba[:, i].ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

In [ ]:
plt.figure()
lw = 2
plt.plot(fpr[1], tpr[1], color='darkorange', lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[1])
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()